# Python Installs

In [ ]:
!pip install datasets
!pip install ai2-olmo
!pip install datasets transformers torch bert_score
!pip install tf-keras
!pip install torch torchvision accelerate
!pip install hf_olmo
!pip install tabulate
!pip install scikit-learn
!pip install sentence-transformers
!pip install bleurt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Load extra, as normal install doesnt work

In [ ]:
!pip install git+https://github.com/google-research/bleurt.git


  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-rtq6log6
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-rtq6log6
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=f797354e023353f9666d6122e9672fe980df600a707f7f8b2f058dfd43c65d8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-5v91t5y4/wheels/64/f4/2c/509a6c31b8ebde891a81029fd94f199b1b92f0e7cfc20d417a
Successfully built BLEURT


# Mount Google Drive

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


# Initialization and Imports

In [ ]:
import pandas as pd
import random
import re
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSequenceClassification

from tabulate import tabulate
from datasets import load_dataset
from bert_score import score as bert_score
from bleurt import score as bleurt_score



# Load Models and Tokenizers

In [ ]:
# Paths to fine-tuned T5, LLaMA, and DeBERTa models
t5_model_path = "/content/drive/My Drive/fine_tuned_flan_t5_base"
llama_model_path = "/content/drive/My Drive/meta_llama_2-7b-hf"
deberta_model_path = "/content/drive/My Drive/fine_tuned_deberta_regression"
bleurt_checkpoint = "/content/drive/My Drive/bleurt-large-512"

# Initialize T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_path)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_path).to('cuda')

# Initialize LLaMA model and tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_path)
llama_model = AutoModelForCausalLM.from_pretrained(
    llama_model_path,
    torch_dtype=torch.float16,

).to('cuda')
llama_tokenizer.pad_token = llama_tokenizer.eos_token  # Ensure tokenizer compatibility

# Initialize DeBERTa model and tokenizer for regression
deberta_tokenizer = AutoTokenizer.from_pretrained(deberta_model_path)
deberta_model = AutoModelForSequenceClassification.from_pretrained(deberta_model_path).to('cuda')
deberta_model.eval()  # Set model to evaluation mode

# Initialize BLEURT scorer
bleurt_scorer = bleurt_score.BleurtScorer(bleurt_checkpoint)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Load Dataset and Preprocessing

In [ ]:
# Load the ACORN dataset and slice the last 150 entries
data = load_dataset("anab/ACORN", split="train")[-100:]
s
# Load a sentence embedding model for similarity comparison
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.16k [00:00<?, ?B/s]

ACORN.jsonl:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3500 [00:00<?, ? examples/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Define Helper Functions

Flan-T5 Explanation Generation

In [ ]:
# Function to generate explanation using Flan-T5
def generate_explanation_t5(question, correct_choice):
    input_text = f"Question: {question} Answer: {correct_choice} Explain why:"
    inputs = t5_tokenizer(input_text, return_tensors="pt").to('cuda')
    outputs = t5_model.generate(**inputs, max_length=100)
    return t5_tokenizer.decode(outputs[0], skip_special_tokens=True)


Few-Shot Example Selection

In [ ]:
# Function to select few-shot examples for LLaMA
def select_few_shot_examples(data, num_examples=5):
    samples = random.sample(list(data), num_examples)
    few_shot_examples = [
        {
            "question": entry["question"],
            "correct_choice": entry["choices"][entry["label"]],
            "explanation": entry["explanation"]
        }
        for entry in samples
    ]
    return few_shot_examples


Prompt Creation for Few-Shot Learning

In [ ]:
# Function to create prompt for few-shot learning for LLaMA
def create_prompt(question, correct_choice, few_shot_examples):
    prompt = ""
    for example in few_shot_examples:
        prompt += (
            f"Question: {example['question']}\n"
            f"Correct Choice: {example['correct_choice']}\n"
            f"Explanation: {example['explanation']}\n\n"
        )
    prompt += (
        f"Question: {question}\n"
        f"Correct Choice: {correct_choice}\n"
        f"Explanation:"
    )
    return prompt


Predict Ratings

In [ ]:
# Function to predict ratings using DeBERTa
def predict_ratings(explanation):
    inputs = deberta_tokenizer(explanation, return_tensors="pt", truncation=True, padding=True, max_length=128).to('cuda')
    
    with torch.no_grad():
        outputs = deberta_model(**inputs)
        predictions = outputs.logits.squeeze(0)  # Get model outputs as tensor
        
        # Ensure predictions are between 0 and 4
        predictions = torch.clamp(predictions, min=0, max=4).tolist()
    
    return {
        "supports": round(predictions[0]),
        "overall": round(predictions[1]),
        "well_written": round(predictions[2]),
        "related": round(predictions[3]),
        "factual": round(predictions[4]),
        "new_info": round(predictions[5]),
        "unnecessary_info": round(predictions[6]),
        "contrastive": round(predictions[7]),
    }


Empty Space for Human Evaluation

In [ ]:
def format_human_evaluation():
    return "\n".join([
        "supports: ",
        "overall: ",
        "well_written: ",
        "related: ",
        "factual: ",
        "new_info: ",
        "unnecessary_info: ",
        "contrastive: "
    ])

Bertscore & Bleurt

In [ ]:
# Function to compute BERTScore
def compute_bertscore(candidate, reference):
    P, R, F1 = bert_score([candidate], [reference], lang="en")
    return F1[0].item()  # Return F1 score for BERTScore

# Function to compute BLEURT score
def compute_bleurt(candidate, reference):
    scores = bleurt_scorer.score(references=[reference], candidates=[candidate])
    return scores[0]  # Return BLEURT score

# Select Data for Evaluation

Restructure Data for correct usage

In [ ]:
restructured_data = [
    {
        key: data[key][i]
        for key in data.keys()
    }
    for i in range(len(data['id']))  # Assuming all lists are of the same length
]

Selecting Amount of Samples & Few Shot Examples

In [ ]:
# Select few-shot examples for LLaMA
few_shot_examples = select_few_shot_examples(restructured_data, num_examples=5)

# Select entries for evaluation
last_samples = restructured_data[-100:]

# Evaluate Model Performance

Ignore Warnings

In [ ]:
transformers_logging.set_verbosity_error()

warnings.filterwarnings(
    "ignore",
    message=r".*Some weights of .* were not initialized from the model checkpoint.*",
    category=UserWarning,
)


Comparing Performance

In [ ]:
final_results = []

for entry in last_samples:
    question = entry['question']
    correct_choice = entry['choices'][entry['label']]
    ground_truth = entry['explanation']
    ground_truth_ratings = entry["voted_ratings"]

    # Generate explanation using Flan-T5
    generated_t5 = generate_explanation_t5(question, correct_choice)
    embeddings_t5 = embedder.encode([generated_t5, ground_truth])
    similarity_t5 = cosine_similarity([embeddings_t5[0]], [embeddings_t5[1]])[0][0]
    ratings_t5 = predict_ratings(generated_t5)
    t5_bertscore = compute_bertscore(generated_t5, ground_truth)
    t5_bleurt = compute_bleurt(generated_t5, ground_truth)

    # Generate explanation using LLaMA
    input_prompt = create_prompt(question, correct_choice, few_shot_examples)
    inputs = llama_tokenizer(input_prompt, return_tensors="pt", padding=True)

    # Move inputs to the same device as the model
    inputs = {key: value.to('cuda') for key, value in inputs.items()}
    outputs = llama_model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2,
    )
    generated_llama = llama_tokenizer.decode(outputs[0], skip_special_tokens=True).split("Explanation:")[-1].strip()
    embeddings_llama = embedder.encode([generated_llama, ground_truth])
    similarity_llama = cosine_similarity([embeddings_llama[0]], [embeddings_llama[1]])[0][0]
    ratings_llama = predict_ratings(generated_llama)
    llama_bertscore = compute_bertscore(generated_llama, ground_truth)
    llama_bleurt = compute_bleurt(generated_llama, ground_truth)


    # Combine results for this entry
    final_results.append({
        "Question": question,
        "Correct Choice": correct_choice,
        "Ground Truth Explanation": ground_truth,
        "Generated Explanation (T5)": generated_t5,
        "Generated Explanation (LLaMA)": generated_llama,
        "Ground Truth Ratings": ground_truth_ratings,
        "T5 Ratings": ratings_t5,
        "T5 Human Ratings 1": format_human_evaluation(),
        "T5 Human Ratings 2": format_human_evaluation(),
        "LLaMA Ratings": ratings_llama,
        "LLaMA Human Ratings 1": format_human_evaluation(),
        "LLaMA Human Ratings 2": format_human_evaluation(),
        "T5 Similarity": round(similarity_t5, 2),
        "LLaMA Similarity": round(similarity_llama, 2),
        "T5 BERTScore": round(t5_bertscore, 2),
        "LLaMA BERTScore": round(llama_bertscore, 2),

        "T5 BLEURT": round(t5_bleurt, 2),
        "LLaMA BLEURT": round(llama_bleurt, 2),

    })

# Format and Display Results

In [ ]:
# Convert results to a pandas DataFrame
df = pd.DataFrame(final_results)

# Function to clean text containing "Question:"
def clean_question_text(text):
    return re.sub(r"(Question:.*|Correct.*)", "", str(text)).strip()

# List of columns to clean
columns_to_clean = [
    "Generated Explanation (T5)",
    "Generated Explanation (LLaMA)"
]

# Apply the cleaning function to each column
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(clean_question_text)


# Format ratings for readability
def format_ratings(ratings):
    return "\n".join([f"{key}: {value}" for key, value in ratings.items()])

df["Ground Truth Ratings"] = df["Ground Truth Ratings"].apply(format_ratings)
df["T5 Ratings"] = df["T5 Ratings"].apply(format_ratings)
df["LLaMA Ratings"] = df["LLaMA Ratings"].apply(format_ratings)

# Select and reorder columns
# Save to CSV
columns_order = [
    "Question",
    "Correct Choice",
    "Ground Truth Explanation",
    "Generated Explanation (T5)",
    "Generated Explanation (LLaMA)",
    "Ground Truth Ratings",
    "T5 Ratings",
    "T5 Human Ratings 1",
    "T5 Human Ratings 2",
    "LLaMA Ratings",
    "LLaMA Human Ratings 1",
    "LLaMA Human Ratings 2",
    "T5 Similarity",
    "LLaMA Similarity",

    "T5 BERTScore",
    "LLaMA BERTScore",
    "T5 BLEURT",
    "LLaMA BLEURT",
]
df = df[columns_order]

# Print the table in the terminal using `tabulate`
print(tabulate(
    df,
    headers="keys",
    tablefmt="grid",
    showindex=False,
))

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------

# Save Results

In [ ]:
# Save to CSV
csv_filename = "generated_explanations_with_ratings.csv"
df.to_csv(csv_filename, index=False)

print(f"\nCSV file '{csv_filename}' has been saved in the current working directory.")



CSV file 'generated_explanations_with_ratings.csv' has been saved in the current working directory.
